In [2]:
# import numpy as np
# testset = np.load("../masif_features/testset.npy", allow_pickle=True)
# valset = np.load("../masif_features/valset.npy", allow_pickle=True)
# trainset = np.load("../masif_features/trainset.npy", allow_pickle=True)
# testset.shape, valset.shape, trainset.shape

In [3]:
# from collections import Counter
# Counter(trainset[:, -1].tolist())

In [ ]:
import torch
from metrics import *
from data_prepare import testloader
from models import (
    GCNN,
    GCNN_desc_pool,
    GCNN_geom_transformer,
    AttGNN_baseline,
    GCNN_geom_transformer_without_descriptors,
    GCNN_geom_transformer_without_graph
)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.cuda("cpu")

# model = GCNN()
# model.load_state_dict(torch.load("../masif_features/GCN_baseline.pth"))

# model = GCNN_desc_pool()
# model.load_state_dict(torch.load("../masif_features/GCN_pool.pth"))

# model = GCNN_geom_transformer(num_layers=2, dropout=0.1, num_features_pro=1024, output_dim=128, descriptor_dim=80, transformer_dim=128, nhead=8, dim_feedforward=256)
# model.load_state_dict(torch.load("../masif_features/GCN_geom_01.pth"))

# model = AttGNN_baseline(dropout=0.1, heads=1)
# model.load_state_dict(torch.load("../masif_features/AttGNN_baseline_l1.pth"))

# model = GCNN_geom_transformer(num_layers=3, dropout=0.1, num_features_pro=1024, output_dim=128, descriptor_dim=80, transformer_dim=128, nhead=8, dim_feedforward=256)
# model.load_state_dict(torch.load("../masif_features/GCN_geom_01_l3.pth"))

model = GCNN_geom_transformer_without_descriptors(num_layers=2, dropout=0.3, num_features_pro=1024, output_dim=128, descriptor_dim=80, transformer_dim=128, nhead=8, dim_feedforward=256)
model.load_state_dict(torch.load("../masif_features/GCNN_geom_transformer_without_descriptors_l2.pth"))

# model = GCNN_geom_transformer_without_graph(num_layers=2, dropout=0.3, num_features_pro=1024, output_dim=128, descriptor_dim=80, transformer_dim=128, nhead=8, dim_feedforward=256)
# model.load_state_dict(torch.load("../masif_features/GCNN_geom_transformer_without_graph_l2.pth"))

model.to(device)
model.eval();

/workspace/ppi_fork/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GCNN_graph_only Loaded


In [7]:
import torch
from tqdm.auto import tqdm

predictions = torch.Tensor()
labels = torch.Tensor()
with torch.no_grad():
    for prot_1, prot_2, label, mas1_straight, mas1_flipped, mas2_straight, mas2_flipped, mas1_straight_lengths, mas1_flipped_lengths, mas2_straight_lengths, mas2_flipped_lengths in tqdm(testloader):
      prot_1 = prot_1.to(device)
      prot_2 = prot_2.to(device)
      mas1_straight = mas1_straight.to(device)
      mas1_flipped = mas1_flipped.to(device)
      mas2_straight = mas2_straight.to(device)
      mas2_flipped = mas2_flipped.to(device)
      output = model(prot_1, prot_2, mas1_straight, mas1_flipped, mas2_straight, mas2_flipped, mas1_straight_lengths, mas1_flipped_lengths, mas2_straight_lengths, mas2_flipped_lengths)
      predictions = torch.cat((predictions, output.cpu()), 0)
      labels = torch.cat((labels, label.view(-1,1).cpu()), 0)
labels = labels.numpy().flatten()
predictions = predictions.numpy().flatten()

  0%|          | 0/7680 [00:00<?, ?it/s]/workspace/ppi_fork/venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:508: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /pytorch/aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(
 56%|█████▌    | 4318/7680 [06:05<03:44, 14.98it/s]

In [ ]:
mse = get_mse(labels, predictions)
acc = get_accuracy(labels, predictions, 0.5)
prec = precision(labels, predictions, 0.5)
sensitivity = sensitivity(labels, predictions,  0.5)
specificity = specificity(labels, predictions, 0.5)
f1 = f_score(labels, predictions, 0.5)
mcc = mcc(labels, predictions,  0.5)
auroc = auroc(labels, predictions)
auprc = auprc(labels, predictions)


print(f'mse : {mse}')
print(f'Accuracy : {acc}')
print(f'precision: {prec}')
print(f'Sensititvity :{sensitivity}')
print(f'specificity : {specificity}')
print(f'f-score : {f1}')
print(f'MCC : {mcc}')
print(f'AUROC: {auroc}')
print(f'AUPRC: {auprc}')

mse : 38.93427658081055
Accuracy : 96.6015625
precision: 0.9592185592185593
Sensititvity :0.9766285430134262
specificity : 0.9543466375068343
f-score : 0.9678452630282125
MCC : 0.9319861396509743
AUROC: 0.9943931259429072
AUPRC: 0.9949231545899945
